In [1]:
# ── Full Reload, Retrain & Back-test (fixed) ────────────────────────
import importlib, pandas as pd
import algo.features as feat_mod
import algo.model    as model_mod
importlib.reload(feat_mod); importlib.reload(model_mod)

from algo.features   import add_indicators, FEATURES
from algo.model      import load_or_train, predict_last, LOOKBACK
from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.backtester import backtest

# 1️⃣ Pull 200 days of 3-min history and build features
cfg    = load_config()
broker = KiteWrapper(cfg)
hist_all = broker.history(days=200, interval="3minute",
                          tradingsymbol="HDFCBANK")
df_all   = add_indicators(hist_all, debug=True).ffill()

# 2️⃣ Split by *calendar days* instead of row count
n_test_days = 20                          # last 20 trading days ≈ 1500 bars
cutoff      = df_all.index[-1] - pd.Timedelta(days=n_test_days)

df_tr = df_all[df_all.index <  cutoff]    # ~180 days
df_te = df_all[df_all.index >= cutoff]    #  last 20 days  (≳ LOOKBACK rows)

print(f"Training rows: {len(df_tr):,}")
print(f"Testing  rows: {len(df_te):,}  (need ≥ {LOOKBACK})")

# 3️⃣ (Re)train the model
model = load_or_train(df_tr, retrain=True)

# 4️⃣ Inspect the pipeline
print("\n▶️ Pipeline steps:", model.named_steps)
gb = model.named_steps["gb"]
print("▶️ Final estimator:", gb)
print("▶️ n_features_in_:", gb.n_features_in_)
print(f"▶️ Expected      : {len(FEATURES)} × {LOOKBACK} = "
      f"{len(FEATURES)*LOOKBACK}\n")

# 5️⃣ Run back-test
trades, metrics = backtest(
    df_te,
    model=model,
    capital=500_000,
    contract_size=1,
    sl_pct=0.0010,
    tp_pct=0.0025,
    trail_pct=0.0010,
    hold_max=10,
    upper=0.78,
    lower=0.63,
    predict_fn=predict_last,
    debug=True,
)

# 6️⃣ Review results
print("Back-test metrics:", metrics)
print("\nFirst few trades:\n", trades.head())


[broker] loaded from /Users/sreejit/PycharmProjects/zerodha-bot/algo/broker.py
[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=200, interval=3minute, symbol=HDFCBANK
[history] range UTC-naive: 2024-12-18 07:43:46.760464 → 2025-07-06 07:43:46.760464
[history] token=341249
[history] got 8750 bars, cursor→2025-03-27 15:30:00
[history] got 8250 bars, cursor→2025-07-04 15:30:00
[history] empty data for 2025-07-04 15:30:00->2025-07-06 07:43:46.760464, breaking loop
[history] complete 17000 bars 2024-12-18 09:15:00 → 2025-07-04 15:27:00
ret1 head: [0.0, 0.002102274278537708, -0.0010353095030515291, -0.003136420662193884, 0.00038302645618459863]
ema_3 head: [1831.35, 1833.275, 1833.2875, 1830.4187499999998, 1829.334375]
ema_8 head: [1831.35, 1832.2055555555555, 1832.4487654320988, 1831.3601508916324, 1830.6690062490475]
vwap head: [1830.55, 1831.9328786325627, 1832.1892438216494, 1831.9381985020973, 1831.6974404207774]
vol_spike head: [0, 0, 0, 0, 0]
Training rows:

In [3]:
trades

,exit_ts,side,entry_price,exit_price,exit_reason,fees,qty,pnl,equity
entry_ts,,,,,,,,,
2025-06-23 11:27:00,2025-06-23 11:57:00,SELL,1947.3,1944.9000,TIME,118.177309,102,126.622691,200126.622691
2025-06-24 10:15:00,2025-06-24 10:45:00,SELL,1972.1,1969.4000,TIME,118.368668,101,154.331332,200280.954023
2025-06-24 11:42:00,2025-06-24 12:03:00,SELL,1975.9,1977.8759,SL,118.642367,101,-318.208267,199962.745756
2025-06-25 11:00:00,2025-06-25 11:30:00,SELL,1980.0,1976.7000,TIME,117.929279,100,212.070720,200174.816477


In [1]:
# ── Full Reload, Retrain & Back-test (fixed) ────────────────────────
import importlib, pandas as pd
import algo.features as feat_mod
import algo.model    as model_mod
importlib.reload(feat_mod); importlib.reload(model_mod)

from algo.features   import add_indicators, FEATURES
from algo.model      import load_or_train, predict_last, LOOKBACK
from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.backtester import backtest

# 1️⃣ Pull 200 days of 3-min history and build features
cfg    = load_config()
broker = KiteWrapper(cfg)
hist_all = broker.history(days=200, interval="3minute",
                          tradingsymbol="IDEA")
df_all   = add_indicators(hist_all, debug=True).ffill()

# 2️⃣ Split by *calendar days* instead of row count
n_test_days = 20                          # last 20 trading days ≈ 1500 bars
cutoff      = df_all.index[-1] - pd.Timedelta(days=n_test_days)

df_tr = df_all[df_all.index <  cutoff]    # ~180 days
df_te = df_all[df_all.index >= cutoff]    #  last 20 days  (≳ LOOKBACK rows)

print(f"Training rows: {len(df_tr):,}")
print(f"Testing  rows: {len(df_te):,}  (need ≥ {LOOKBACK})")

# 3️⃣ (Re)train the model
model = load_or_train(df_tr, retrain=False)

# 4️⃣ Inspect the pipeline
print("\n▶️ Pipeline steps:", model.named_steps)
gb = model.named_steps["gb"]
print("▶️ Final estimator:", gb)
print("▶️ n_features_in_:", gb.n_features_in_)
print(f"▶️ Expected      : {len(FEATURES)} × {LOOKBACK} = "
      f"{len(FEATURES)*LOOKBACK}\n")

# 5️⃣ Run back-test
trades, metrics = backtest(
    df_te,
    model=model,
    capital=500_000,
    contract_size=1,
    sl_pct=0.0015,
    tp_pct=0.0025,
    trail_pct=0.0010,
    hold_max=10,
    upper=0.481,
    lower=0.627,
    predict_fn=predict_last,
    debug=True,
)

# 6️⃣ Review results
print("Back-test metrics:", metrics)
print("\nFirst few trades:\n", trades.head())


[broker] loaded from /Users/sreejit/PycharmProjects/zerodha-bot/algo/broker.py
[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=200, interval=3minute, symbol=IDEA
[history] range UTC-naive: 2024-12-18 07:51:43.525921 → 2025-07-06 07:51:43.525921
[history] token=3677697
[history] got 8750 bars, cursor→2025-03-27 15:30:00
[history] got 8250 bars, cursor→2025-07-04 15:30:00
[history] empty data for 2025-07-04 15:30:00->2025-07-06 07:51:43.525921, breaking loop
[history] complete 17000 bars 2024-12-18 09:15:00 → 2025-07-04 15:27:00
ret1 head: [0.0, 0.0012820512820512775, -0.002560819462227859, 0.005134788189987249, -0.0012771392081736277]
ema_3 head: [7.8, 7.805, 7.797499999999999, 7.81375, 7.816875]
ema_8 head: [7.8, 7.8022222222222215, 7.799506172839505, 7.806282578875171, 7.809330894680689]
vwap head: [7.796666666666667, 7.801505480949821, 7.801190345451425, 7.804379032579051, 7.8058965059929895]
vol_spike head: [0, 0, 0, 0, 0]
Training rows: 15,125
Testing  

In [4]:
import datetime as dt
import pandas as pd
from algo.config import load_config
from algo.broker import KiteWrapper

# ── USER INPUT ──────────────────────────────────────────────────────
SYMBOL      = "HDFCBANK"
INTERVAL    = "3minute"
TARGET_DATE = dt.date(2025, 7, 3)                    # the day you want
OUT_FILE    = f"{SYMBOL}_{TARGET_DATE}_{INTERVAL}.csv"

# ── 1) Compute how many days back to pull so TARGET_DATE is included ─
today        = dt.date.today()
days_back    = (today - TARGET_DATE).days + 1        # +1 so the window covers it
# e.g. if today=2025-07-06 and TARGET_DATE=2025-07-03 → days_back=4

# ── 2) Fetch a little more than you need, then filter by the exact date ─
cfg    = load_config()
wrap   = KiteWrapper(cfg)
hist   = wrap.history(
    days=days_back,
    interval=INTERVAL,
    tradingsymbol=SYMBOL,
)

# ── 3) Filter to the single calendar date ───────────────────────────────
df = hist.loc[hist.index.date == TARGET_DATE]

# ── 4) Save & report ───────────────────────────────────────────────────
df.to_csv(OUT_FILE)
print(f"Requested {TARGET_DATE}, pulled {len(hist)} bars, saved {len(df)} bars → {OUT_FILE}")


[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=4, interval=3minute, symbol=HDFCBANK
[history] range UTC-naive: 2025-07-02 08:29:19.273875 → 2025-07-06 08:29:19.273875
[history] token=341249
[history] got 375 bars, cursor→2025-07-04 15:30:00
[history] empty data for 2025-07-04 15:30:00->2025-07-06 08:29:19.273875, breaking loop
[history] complete 375 bars 2025-07-02 09:15:00 → 2025-07-04 15:27:00
Requested 2025-07-03, pulled 375 bars, saved 125 bars → HDFCBANK_2025-07-03_3minute.csv
